In [23]:
import os
os.environ['MY_TECTON_API_KEY']='<INSERT YOUR KEY HERE>'
os.environ['MY_LLM_API_KEY']='<INSERT YOUR KEY HERE>'

import requests, json, os
def get_online_feature_data(user_id, workspace_name="nl_demo_apply", fs_name="user_service:v1", context_map=None):
    tecton_api_key = os.getenv('MY_TECTON_API_KEY')

    url = "https://community.tecton.ai/api/v1/feature-service/get-features"

    # Define the request payload
    payload = {
        "params": {
            "feature_service_name": fs_name,
            "join_key_map": {
                "user_id": user_id
            },
            "workspace_name": workspace_name,
            "metadata_options": {"include_names": True}
        }
    }

    if context_map:
    # Create the request_context_map field using a dictionary comprehension
        payload["params"]["request_context_map"] = {
        field_name: context_map[field_name] for field_name in context_map
    }

    headers = {
    "Authorization": f"Tecton-key {tecton_api_key}",
    "Content-Type": "application/json"
    }

    # Send the POST request
    response = requests.post(url, json=payload, headers=headers)

    dict_response = json.loads(response.text)

    online_features_map = {}
    try: 
        for i, field_name in enumerate(dict_response['metadata']['features']):
            online_features_map[field_name['name']] = dict_response['result']['features'][i]
    except KeyError as e: 
        print(dict_response)

    return dict_response, online_features_map

import json
from urllib import request, error
import time

def get_travel_recommendation(context):
    # Replace 'your_api_endpoint' with the actual GPT API endpoint you're using.
    # Replace 'your_api_key' with your actual API key.
    api_endpoint = 'https://api.openai.com/v1/chat/completions'
    api_key = os.getenv('MY_LLM_API_KEY')
    data = {
       "model": "gpt-4",
       "messages": [
           {
               "role": "system",
               "content": context }
       ]
    }

    # Convert the data dictionary to a JSON string.
    data_json = json.dumps(data).encode('utf-8')

    # Prepare the request with the necessary headers.
    req = request.Request(api_endpoint, data=data_json)
    req.add_header('Content-Type', 'application/json')
    req.add_header('Authorization', f'Bearer {api_key}')

    max_retries = 3
    delay = 1
    # Function to make the API call with retry logic

    for attempt in range(max_retries):
        try:
            # Make the request to the API
            with request.urlopen(req) as response:
                # Read and decode the response
                response_body = response.read().decode('utf-8')
                return json.loads(response_body)
        except error.HTTPError as e:
            # You can add more sophisticated error handling here
            print(f"HTTPError: {e.code}")
            print(f"Content: {e.read().decode('utf-8')}")
        except error.URLError as e:
            print(f"URLError: {e.reason}")

        # Wait for a specified delay before retrying
        print(f"Retrying in {delay} seconds...")
        time.sleep(delay)
        return None #no response from API

def your_own_recommendation_model(features_dictionary):
    #This is where your ML model would analyze the features and compose some candidates to recommend
    #for the demo we return a list of 10 candidates 
    return ["Amsterdam, Netherlands", "Kyoto, Japan", "Melbourne, Australia", "Copenhagen, Denmark", "Barcelona, Spain", "Portland, Oregon, USA", "Vancouver, Canada", "Chiang Mai, Thailand", "Berlin, Germany", "Buenos Aires, Argentina"]

In [24]:
response, features_dictionary = get_online_feature_data("demouser123", fs_name="user_service:v1")
feature_context = []
for feature_name in features_dictionary: 
    feature_context.append(feature_name + ": " + str(features_dictionary[feature_name]))

print(feature_context)

['user_flight_search_fv.departure_date: 2023-12-13', 'user_flight_search_fv.destination_name: Singapore Changi Airport', 'user_flight_search_fv.origin_name: San Francisco International Airport', 'user_flight_search_fv.return_date: 2023-12-27', 'user_home_airport_fv.home_airport_code: SFO', 'user_home_airport_fv.home_airport_name: San Francisco International Airport', "user_last_trip_completed_fv.destination_name_last_5_365d_1d: ['Narita International Airport']", "user_likes_fv.favorite_activity: ['Bicycling', 'Running', 'Food']"]


In [25]:
candidate_list = your_own_recommendation_model(features_dictionary)
feature_context.append("recommended_cities: " + str(candidate_list))

In [26]:
print(feature_context)

['user_flight_search_fv.departure_date: 2023-12-13', 'user_flight_search_fv.destination_name: Singapore Changi Airport', 'user_flight_search_fv.origin_name: San Francisco International Airport', 'user_flight_search_fv.return_date: 2023-12-27', 'user_home_airport_fv.home_airport_code: SFO', 'user_home_airport_fv.home_airport_name: San Francisco International Airport', "user_last_trip_completed_fv.destination_name_last_5_365d_1d: ['Narita International Airport']", "user_likes_fv.favorite_activity: ['Bicycling', 'Running', 'Food']", "recommended_cities: ['Amsterdam, Netherlands', 'Kyoto, Japan', 'Melbourne, Australia', 'Copenhagen, Denmark', 'Barcelona, Spain', 'Portland, Oregon, USA', 'Vancouver, Canada', 'Chiang Mai, Thailand', 'Berlin, Germany', 'Buenos Aires, Argentina']"]


In [27]:
context1 = ("You are a travel agent for my travel website. Use the following context to make a relevant travel recommendation. Constraints: "
                          "1. You must select an option from the list of recommended cities provided."
                          "2. The recommendation should be based on the place they searched for before, be similar city not far from their search. "
                          "3. Include a suggested itinerary of things to do and see. The itinerary should include at least 3 things. "
                          "4. No lodging recommendations. Focus on things to do and see. "
                          "5. Return your response in valid JSON only which can easily be parsed with json.loads() in Python. No text outside of the JSON: "
                          "{'recommendation_text':<your full recommendation text here>, "
                          "'recommendation_airport': <IATA code for airport for your recommendation>, "
                          "'recommendation_destination': <the city you are recommending> "
                          "'explanation':<explain why you recommended what you did>}"
                          "Here is the user context along with recommended cities: ")

context1 = context1 + ", ".join(feature_context)

print(context1)


You are a travel agent for my travel website. Use the following context to make a relevant travel recommendation. Constraints: 1. You must select an option from the list of recommended cities provided.2. The recommendation should be based on the place they searched for before, be similar city not far from their search. 3. Include a suggested itinerary of things to do and see. The itinerary should include at least 3 things. 4. No lodging recommendations. Focus on things to do and see. 5. Return your response in valid JSON only which can easily be parsed with json.loads() in Python. No text outside of the JSON: {'recommendation_text':<your full recommendation text here>, 'recommendation_airport': <IATA code for airport for your recommendation>, 'recommendation_destination': <the city you are recommending> 'explanation':<explain why you recommended what you did>}Here is the user context along with recommended cities: user_flight_search_fv.departure_date: 2023-12-13, user_flight_search_fv.

In [28]:
result = get_travel_recommendation(context1)
LLM_response = json.loads(result['choices'][0]['message']['content'])
print(LLM_response)

{'recommendation_text': 'Based on your previous trip and interests, we recommend your next destination as Kyoto, Japan. Kyoto is a fusion of ancient and modern, just like Singapore. As a fan of bicycling, you can explore the city via the Kamo River cycling path, which is favored among locals and tourists. Known for its rich food culture, explore Nishiki Market where an array of Japanese street food is waiting for you. Visit the famous Fushimi Inari Shrine to add a historical aspect to your trip.', 'recommendation_airport': 'KIX', 'recommendation_destination': 'Kyoto, Japan', 'explanation': 'Kyoto was chosen due to its relative geographical proximity to your last destination and alignment with your food and biking interests. Additionally, like Singapore, Kyoto offers a fusion of modern and traditional experiences.'}


In [29]:
overall_recommendation = {}
overall_recommendation["recommendation"] = LLM_response["recommendation_text"]
overall_recommendation["explanation"] = LLM_response["explanation"]

In [33]:
response, features_dictionary = get_online_feature_data("demouser123", fs_name = "hotel_and_flight_service:v1", context_map=LLM_response)
feature_context = []
for feature_name in features_dictionary: 
    feature_context.append(feature_name + ": " + str(features_dictionary[feature_name]))
print(feature_context)

["get_best_hotels.hotel_recommendations: ['Hotel Mume located at 東山区新門前通梅本町261 in Kyoto, 26. Average rating: 5.0 (based on 8 reviews) Yelp URL: https://www.yelp.com/biz/%E3%83%9B%E3%83%86%E3%83%AB-%E3%83%A0%E3%83%A1-%E4%BA%AC%E9%83%BD%E5%B8%82', 'Shiraume located at 東山区祇園新橋白川畔 in Kyoto, 26. Average rating: 5.0 (based on 12 reviews) Yelp URL: https://www.yelp.com/biz/%E7%99%BD%E6%A2%85-%E4%BA%AC%E9%83%BD%E5%B8%82', 'HOTEL GRACERY KYOTO SANJO located at 中京区六角通寺町東入桜之町420番 in Kyoto, 26. Average rating: 5.0 (based on 8 reviews) Yelp URL: https://www.yelp.com/biz/%E3%83%9B%E3%83%86%E3%83%AB%E3%82%B0%E3%83%AC%E3%82%A4%E3%82%B9%E3%83%AA%E3%83%BC%E4%BA%AC%E9%83%BD%E4%B8%89%E6%9D%A1-%E4%BA%AC%E9%83%BD%E5%B8%82-2']", 'get_booking_url.booking_url: https://www.kayak.com/flights/SFO-KIX/2023-12-13/2023-12-27?sort=bestflight_a']


In [ ]:
context2 = ("You are a travel agent for my travel website. We generated a list of recommended hotels and a flight booking link based on your recent recommendation. Constraints: "
                          "1. Recommend each hotel and include some interesting facts or points of interest about them if available. "
                          "2. Include a call to action to book a flight with the booking link. "
                          "3. Return your response in valid JSON only which can easily be parsed with json.loads() in Python. No text outside of the JSON: "
                          "{'recommendation_text':<your full recommendation text here>}"
                          "Here is the context: ")

context2 = context2 + ", ".join(feature_context)

In [ ]:
response = get_travel_recommendation(context2)
LLM_response_2 = json.loads(response['choices'][0]['message']['content'])

print(LLM_response_2)

In [ ]:
overall_recommendation["hotel_and_flight_rec"] = LLM_response_2["recommendation_text"]
overall_recommendation["flight_booking_url"] = features_dictionary["get_booking_url.booking_url"]

for key in overall_recommendation: 
    print(str(key).upper() + ": " + overall_recommendation[key] + "\n")

In [ ]:
#putting it all together

context1 = ("You are a travel agent for my travel website. Use the following context to make a relevant travel recommendation. Constraints: "
                          "1. You must select an option from the list of recommended cities provided."
                          "2. The recommendation should be based on the place they searched for before, be similar city not far from their search. "
                          "3. Include a suggested itinerary of things to do and see. The itinerary should include at least 3 things. "
                          "4. No lodging recommendations. Focus on things to do and see. "
                          "5. Return your response in valid JSON only which can easily be parsed with json.loads() in Python. No text outside of the JSON: "
                          "{'recommendation_text':<your full recommendation text here>, "
                          "'recommendation_airport': <IATA code for airport for your recommendation>, "
                          "'recommendation_destination': <the city you are recommending> "
                          "'explanation':<explain why you recommended what you did>}"
                          "Here is the user context along with recommended cities: ")

context2 = ("You are a travel agent for my travel website. We generated a list of recommended hotels and a flight booking link based on your recent recommendation. Constraints: "
                          "1. Recommend each hotel and include some interesting facts or points of interest about them if available. "
                          "2. Include a call to action to book a flight with the booking link. "
                          "3. Return your response in valid JSON only which can easily be parsed with json.loads() in Python. No text outside of the JSON: "
                          "{'recommendation_text':<your full recommendation text here>}"
                          "Here is the context: ")

user_list = ["demouser123", "9tb07o5m"] #
recommendations = [] 
for user in user_list: 
    overall_recommendation = {}

    #Get initial recommendation and explanation
    _, features_dictionary = get_online_feature_data(user, fs_name="user_service:v1")
    feature_context = [f"{feature_name}: {features_dictionary[feature_name]}" for feature_name in features_dictionary] #feature strings
    feature_context.append("recommended_cities: " + str(your_own_recommendation_model(features_dictionary))) #append recommendation candidates
    context_combined = context1 + ", ".join(feature_context)
    LLM_response = json.loads(get_travel_recommendation(context_combined)['choices'][0]['message']['content'])

    #Get hotel and flight booking recommendation
    _, features_dictionary2 = get_online_feature_data(user, fs_name = "hotel_and_flight_service:v1", context_map=LLM_response)
    feature_context2 = [f"{feature_name}: {features_dictionary2[feature_name]}" for feature_name in features_dictionary2] #feature strings
    context_combined_2 = context2 + ", ".join(feature_context2)
    LLM_response_2 = json.loads(get_travel_recommendation(context_combined_2)['choices'][0]['message']['content'])

    overall_recommendation["recommendation"] = LLM_response["recommendation_text"]
    overall_recommendation["explanation"] = LLM_response["explanation"]
    overall_recommendation["hotel_and_flight_rec"] = LLM_response_2["recommendation_text"]
    overall_recommendation["flight_booking_url"] = features_dictionary2["get_booking_url.booking_url"]

    recommendations.append(overall_recommendation)



In [ ]:
user_recs = zip(user_list, recommendations)
for item in list(user_recs): 
    print("user_id: " + item[0])
    for key in item[1]: 
        print(str(key).upper() + ": " + item[1][key] + "\n")